In [22]:
import numpy as np
from sklearn import metrics
import pandas as pd
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Layer, Input
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [23]:
# Load train features data
X_train = np.load('data/X_train.npy')
X_train = X_train[:10000,:]
X_train.shape

(10000, 1000, 4)

In [24]:
# Load train labels data
y_train = np.load('data/y_train.npy')
y_train = y_train[:10000,:]
y_train.shape

(10000, 690)

In [25]:
# Load test features data
X_test = np.load('data/X_test.npy')
X_test = X_test[:1000,:]
X_test.shape

(1000, 1000, 4)

In [26]:
# Load test labels
y_test = np.load('data/y_test.npy')
y_test = y_test[:1000,:]
y_test.shape

(1000, 690)

In [27]:
input_data = Input(shape=(1000,4))

# Convolutional Layer
output = Conv1D(320,kernel_size=26,activation="relu")(input_data)
output = MaxPooling1D()(output)
output = Dropout(0.2)(output)

#BiLSTM Layer
output = Bidirectional(LSTM(320,return_sequences=True))(output)
output = Dropout(0.5)(output)

flat_output = Flatten()(output)

#FC Layer
FC_output = Dense(695)(flat_output)
FC_output = Activation('relu')(FC_output)

#Output Layer
output = Dense(690)(FC_output)
output = Activation('sigmoid')(output)

model = Model(inputs=input_data, outputs=output)

print('compiling model')
model.compile(loss='binary_crossentropy', optimizer='adam')

print('model summary')
model.summary()

checkpointer = ModelCheckpoint(filepath="./model/bilstm_model.hdf5", verbose=1, save_best_only=False)
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

history = model.fit(X_train, y_train, batch_size=100, epochs=2, shuffle=True, verbose=1, validation_split=0.1, callbacks=[checkpointer,earlystopper])

compiling model
model summary
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1000, 4)]         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 975, 320)          33600     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 487, 320)         0         
 1D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 487, 320)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 487, 640)         1640960   
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         

In [28]:
np.mean(history.history['loss'])

0.07223836332559586

In [29]:
np.mean(history.history['val_loss'])

0.059938663616776466